## Part 1: Preprocessing

In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [24]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [25]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [26]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 
            'BusinessTravel', 
            'DistanceFromHome', 
            'Education', 
            'EducationField', 
            'EnvironmentSatisfaction', 
            'HourlyRate', 
            'JobInvolvement',
            'JobLevel',
            'JobRole',
            'JobSatisfaction',
            'MaritalStatus',
            'NumCompaniesWorked',
            'OverTime',
            'PercentSalaryHike',
            'PerformanceRating',
            'RelationshipSatisfaction',
            'StockOptionLevel',
            'TotalWorkingYears',
            'TrainingTimesLastYear',
            'WorkLifeBalance',
            'YearsAtCompany',
            'YearsInCurrentRole',
            'YearsSinceLastPromotion',
            'YearsWithCurrManager']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                          int64
BusinessTravel              object
DistanceFromHome             int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

In [27]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [28]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

cols_to_encode = ['BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']

# Determine the number of value counts in each column.
for col in cols_to_encode:
    print(f'{col}: {X_train[col].value_counts()}\n')


BusinessTravel: BusinessTravel
Travel_Rarely        781
Travel_Frequently    210
Non-Travel           111
Name: count, dtype: int64

EducationField: EducationField
Life Sciences       448
Medical             345
Marketing           127
Technical Degree    100
Other                61
Human Resources      21
Name: count, dtype: int64

JobRole: JobRole
Sales Executive              248
Research Scientist           219
Laboratory Technician        196
Manufacturing Director       103
Healthcare Representative     95
Manager                       82
Sales Representative          61
Research Director             61
Human Resources               37
Name: count, dtype: int64

MaritalStatus: MaritalStatus
Married     500
Single      356
Divorced    246
Name: count, dtype: int64

OverTime: OverTime
No     799
Yes    303
Name: count, dtype: int64



In [29]:
# Use OneHotEncoder to encode the columns
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')


# Fit and encode the training data
X_train_encoded = encoder.fit_transform(X_train[cols_to_encode])
X_test_encoded = encoder.transform(X_test[cols_to_encode])

# Convert the encoded arrays to DataFrames
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(cols_to_encode))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(cols_to_encode))

print(X_train_encoded_df.dtypes)
print(X_test_encoded_df.dtypes)


BusinessTravel_Travel_Frequently    float64
BusinessTravel_Travel_Rarely        float64
EducationField_Life Sciences        float64
EducationField_Marketing            float64
EducationField_Medical              float64
EducationField_Other                float64
EducationField_Technical Degree     float64
JobRole_Human Resources             float64
JobRole_Laboratory Technician       float64
JobRole_Manager                     float64
JobRole_Manufacturing Director      float64
JobRole_Research Director           float64
JobRole_Research Scientist          float64
JobRole_Sales Executive             float64
JobRole_Sales Representative        float64
MaritalStatus_Married               float64
MaritalStatus_Single                float64
OverTime_Yes                        float64
dtype: object
BusinessTravel_Travel_Frequently    float64
BusinessTravel_Travel_Rarely        float64
EducationField_Life Sciences        float64
EducationField_Marketing            float64
EducationField_Med

In [30]:
X_train.columns

Index(['Age', 'BusinessTravel', 'DistanceFromHome', 'Education',
       'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [31]:
# Create a list of columns to scale
cols_to_scale = ['Age', 
                'DistanceFromHome',
                'Education',
                'EnvironmentSatisfaction', 
                'HourlyRate',
                'JobInvolvement',
                'JobLevel',
                'JobSatisfaction', 
                'NumCompaniesWorked', 
                'PercentSalaryHike',
                'PerformanceRating',
                'RelationshipSatisfaction',
                'StockOptionLevel', 
                'TotalWorkingYears', 
                'TrainingTimesLastYear',
                'WorkLifeBalance', 
                'YearsAtCompany', 
                'YearsInCurrentRole', 
                'YearsSinceLastPromotion', 
                'YearsWithCurrManager']

In [32]:
# Create a StandardScaler
scale = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scale.fit_transform(X_train[cols_to_scale])


# Scale the training and testing data
X_test_scaled = scale.transform(X_test[cols_to_scale])

# Convert the scaled arrays to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=cols_to_scale)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=cols_to_scale)

X_train_scaled_df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,-0.194711,-1.009847,0.095106,-0.652015,0.627536,0.356004,-0.959999,-1.533156,0.555189,-0.593901,-0.431534,1.183383,0.238964,-0.815651,-0.626466,-1.079249,-0.657409,-0.624072,-0.367435,-0.597977
1,-0.960259,1.237833,1.074606,1.183287,0.578278,-1.060302,-0.959999,-1.533156,-0.665193,-0.864907,-0.431534,0.252556,2.569394,-0.685232,-0.626466,0.336623,-0.169006,0.205181,0.247935,-0.319813
2,-1.944534,-1.009847,-1.863894,0.265636,1.661972,-2.476608,-0.959999,-1.533156,-1.071988,1.574145,2.317314,-1.609097,-0.926251,-1.337327,1.716403,1.752495,-1.145811,-1.176907,-0.675120,-1.154305
3,0.352108,0.863220,1.074606,-1.569667,-0.751712,-1.060302,-0.064217,0.266139,1.368778,-0.322895,-0.431534,1.183383,-0.926251,-0.163556,2.497360,0.336623,-0.983010,-1.176907,-0.675120,-1.154305
4,0.461472,-1.009847,-0.884394,-0.652015,1.366419,0.356004,-0.064217,1.165786,2.182366,-1.135913,-0.431534,-1.609097,-0.926251,-0.424394,-2.188379,-2.495122,-0.169006,-0.071237,-0.675120,0.236515


In [33]:
# Concatenate the scaled and encoded columns
X_train_df = pd.concat([X_train_encoded_df, X_train_scaled_df], axis=1)
X_test_df = pd.concat([X_test_encoded_df, X_test_scaled_df], axis=1)

In [34]:
y_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [35]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
dept_encoder.fit(np.array(y_train['Department'].values.reshape(-1, 1)))

# Create two new variables by applying the encoder
# to the training and testing data
y_dept_train = dept_encoder.transform(np.array(y_train['Department'].values.reshape(-1, 1)))
y_dept_test = dept_encoder.transform(np.array(y_test['Department'].values.reshape(-1, 1)))



In [36]:
y_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [37]:
# Create a OneHotEncoder for the Attrition column
atr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
# atr_encoder.fit(np.array(y_train['Attrition'].values.reshape(-1, 1)))
atr_encoder.fit(np.array(y_train['Attrition'].values.reshape(-1, 1)))

# Create two new variables by applying the encoder
# to the training and testing data
y_atr_train = atr_encoder.transform(np.array(y_train['Attrition'].values.reshape(-1, 1)))
y_atr_test = atr_encoder.transform(np.array(y_test['Attrition'].values.reshape(-1, 1)))
y_atr_train


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [38]:
X_train_df.shape[0]

1102

In [39]:
# Find the number of columns in the X training data
number_input_features = len(X_train_df.columns)
# number_input_features = X_train_df.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(128, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu', name='shared_layer2')(shared_layer1)
shared_layer3 = layers.Dense(32, activation='relu', name='shared_layer3')(shared_layer2)

In [40]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden_layer = layers.Dense(16, activation='relu', name='dept_hidden_layer')(shared_layer3)

# Create the output layer
dept_output_layer = layers.Dense(y_dept_train.shape[1], activation='softmax', name='dept_output_layer')(dept_hidden_layer)


In [41]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
atr_hidden_layer = layers.Dense(16, activation='relu', name='atr_hidden_layer')(shared_layer3)

# Create the output layer
atr_output_layer = layers.Dense(y_atr_train.shape[1], activation='sigmoid', name='atr_output_layer')(atr_hidden_layer)


In [42]:
# Create the model
model = Model(
    inputs=input_layer, 
    outputs=[dept_output_layer, atr_output_layer],
    name='model')

# Compile the model
model.compile(optimizer='adam',
                loss={'dept_output_layer': 'categorical_crossentropy',
                    'atr_output_layer': 'categorical_crossentropy'},
                metrics={'dept_output_layer': 'accuracy',
                    'atr_output_layer': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 38)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 128)       │      4,992 │ input[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 64)        │      8,256 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer3       │ (None, 32)        │      2,080 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_hidden_layer   │ (None, 16)        │        528 │ shared_layer3[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ atr_hidden_layer    │ (None, 16)        │        528 │ shared_layer3[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output_layer   │ (None, 3)         │         51 │ dept_hidden_laye… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ atr_output_layer    │ (None, 2)         │         34 │ atr_hidden_layer… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,469 (64.33 KB)

 Trainable params: 16,469 (64.33 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# Train the model
model.fit(
        X_train_df, 
        {'dept_output_layer': y_dept_train, 
        'atr_output_layer': y_atr_train}, 
        epochs=100, 
        batch_size=32,
        validation_split=0.2
        )


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - atr_output_layer_accuracy: 0.8499 - dept_output_layer_accuracy: 0.6438 - loss: 1.4319 - val_atr_output_layer_accuracy: 0.8371 - val_dept_output_layer_accuracy: 0.6380 - val_loss: 1.1545
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - atr_output_layer_accuracy: 0.8486 - dept_output_layer_accuracy: 0.6701 - loss: 1.0407 - val_atr_output_layer_accuracy: 0.8371 - val_dept_output_layer_accuracy: 0.8054 - val_loss: 0.9690
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - atr_output_layer_accuracy: 0.8429 - dept_output_layer_accuracy: 0.8606 - loss: 0.8502 - val_atr_output_layer_accuracy: 0.8371 - val_dept_output_layer_accuracy: 0.9186 - val_loss: 0.7821
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - atr_output_layer_accuracy: 0.8535 - dept_output_layer_accuracy: 0.9268 - loss: 0.6571 - val_atr_output_layer_accuracy: 0.8462 - val_dept_output_layer_accuracy: 0.9457 - val_loss: 0.6308
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

In [44]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_df, 
            {'dept_output_layer': y_dept_test, 
            'atr_output_layer': y_atr_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - atr_output_layer_accuracy: 0.8175 - dept_output_layer_accuracy: 0.9686 - loss: 2.3720


[2.113483190536499, 0.8125, 0.9619565010070801]

In [25]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]*100}")
print(f"Attrition Accuracy: {test_results[1]*100}")

Department Accuracy: 98.097825050354
Attrition Accuracy: 86.1413061618805


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy is not the best metric to use on this data given the imbalance identified by the value counts for Department and Attrition features. The model accuracy is      measuring the proportion of correctly predicated instances out of total instances. A higher probability for one class may result in a high accuracy score by simply predicting the majority class, even though the model may not be correctly identifying the minority class.

2. For the dept_output_layer, the softmax activation function was chosen because it is appropriate for multi-class classification problems. The softmax function converts the raw output scores into probabilities that sum to 1, allowing the model to make a probabilistic prediction for each class, and the class with the highest probability is chosen as the predicted class.
For the atr_output_layer, the sigmoid activation function was chosen because it is appropriate for binary classification problems. The sigmoid function outputs a probability value between 0 and 1, which can be interpreted as the likelihood of the positive class. A threshold (usually 0.5) is then used to decide the class label.

3. The model score can be improved through the following ways:
Data Preprocessing - Adjusting the class weights or using resampling techniques to address the imbalance in the minority class. We could also creating or transforming features to provide a better way to capture the underlying patterns in the data.
Model Architecture - Changing the model architecture by adding more layers, increasing the number of neurons in the layers, or using a different activation function.
Hyperparameter Tuning - Changing the hyperparameters such as the learning rate, batch size, optimizer, loss function, and number of epochs.